In [ ]:
import os
import glob
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2
from PIL import Image

from ultralytics import YOLO

## Constants

In [ ]:
%matplotlib inline

In [ ]:
WIDTH = 700

In [ ]:
HEIGHT = 500

In [ ]:
IMAGE_INDEX = 7

In [ ]:
IOU_THRESHOLD = 0.5

In [ ]:
CONF_THRESHOLD = 0.6

## Load Model

In [ ]:
model = YOLO('~/master_project/weights/yolo11m.pt')

## Load Dataset

In [ ]:
PREFIX = '../data/SSLAD-2D/labeled/train'
ANNOTATIONS_PREFIX = '../data/SSLAD-2D/labeled/annotations'

In [ ]:
files = os.listdir(PREFIX)
files[:5]

In [ ]:
# from IPython.display import Image, display

# display(Image(filename=os.path.join(PREFIX, files[0]), width=WIDTH, height=HEIGHT))

In [ ]:
annos = os.listdir(ANNOTATIONS_PREFIX)

In [ ]:
with open(os.path.join(ANNOTATIONS_PREFIX, 'instance_train.json'), 'r') as fp:
    train_annos_str = fp.read()

In [ ]:
train_annos = json.loads(train_annos_str)

In [ ]:
train_annos

## Predict

In [ ]:
test_image_path = os.path.join(PREFIX, files[IMAGE_INDEX])
test_image_path

In [ ]:
parts = test_image_path.split('_')
image_id = int(parts[2].lstrip('0'))
image_id

In [ ]:
im = Image.open(test_image_path)

In [ ]:
np_im = np.array(im)

In [ ]:
results = model(test_image_path)

In [ ]:
for box in results[0].boxes:
    print(box.xyxy.tolist()[0])
    x1, y1, x2, y2 = map(int, box.xyxy.tolist()[0])
    class_id = int(box.cls)
    class_name = results[0].names[class_id]
    confidence = box.conf.item()
    
    # Draw rectangle and label on the frame
    cv2.rectangle(np_im, (x1, y1), (x2, y2), (0, 255, 0), 2)
    label = f"{class_name}: {confidence:.2f}"
    cv2.putText(np_im, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

In [ ]:
plt.figure(figsize=(10,15))
plt.imshow(np_im)

In [ ]:
im2 = Image.open(test_image_path)
np_im2 = np.array(im2)

In [ ]:
for ann in train_annos['annotations']:
    if(ann['image_id'] == image_id):
        print(ann)

        bbox = ann['bbox'] # x, y, w, h

        x1, y1, w, h = bbox

        x2, y2 = x1 + w, y1 + h
        
        cv2.rectangle(np_im2, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = f"{class_name}"
        cv2.putText(np_im2, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

In [ ]:
plt.figure(figsize=(10,15))
plt.imshow(np_im2)